In [ ]:
import requests
from requests.auth import HTTPBasicAuth

# Fetch flows information from the ONOS controller
response = requests.get("http://localhost:8181/onos/v1/flows/of%3A0000000000000001", auth=("onos", "rocks"))
flows = response.json()

# Define the IP address for filtering
ip = "10.0.0.1"

# Filter flow IDs based on specific criteria
filtered_flow_ids = [
    entry['id'] for entry in flows.get("flows", [])
    if entry.get("appId") == "org.onosproject.rest"
    and any(
        criterion.get("type") == "IPV4_SRC" and criterion.get("ip") == "10.0.0.1/32"
        for criterion in entry.get("selector", {}).get("criteria", [])
    )
    and any(
        instruction.get("type") == "OUTPUT"
        for instruction in entry.get("treatment", {}).get("instructions", [])
    )
]

# Set headers for the DELETE requests
headers = {'Accept': 'application/json'}
base_url = "http://localhost:8181/onos/v1/flows/of%3A0000000000000001/"

# Iterate through filtered flow IDs and send DELETE requests
for flow_id in filtered_flow_ids:
    url = f"{base_url}{flow_id}"
    response = requests.delete(url, headers=headers, auth=("onos", "rocks"))
    print(response)  # Print the response for each delete request